# Megaline: Smart o Ultra

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

# Descripción de los datos

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

- `сalls` — número de llamadas,
- `minutes` — duración total de la llamada en minutos,
- `messages` — número de mensajes de texto,
- `mb_used` — Tráfico de Internet utilizado en MB,
- `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).


# Inicialización

In [84]:
# Carga todas las librerías
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

# Cargar los datos

In [70]:
# Carga los datos en DataFrames
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except Exception:
    df = pd.read_csv('datasets/users_behavior.csv')

In [71]:
# Imprime la información general/resumen sobre el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [72]:
# Imprime una muestra aleatoria de 5 filas del DataFrame
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
3055,36.0,206.01,1.0,14830.67,0
1435,76.0,585.56,30.0,22165.13,0
583,41.0,227.58,0.0,20292.33,0
2961,137.0,899.90,0.0,10289.26,1
1858,185.0,1217.83,17.0,1444.81,1


# Segmentar los datos

In [73]:
# Separar los datos en características (features) y objetivo (target)
features = df.drop("is_ultra", axis=1)
target = df["is_ultra"]

In [74]:
# Separar los datos en subconjuntos de entrenamiento, validación y prueba (3:1:1)
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target, test_size=.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, test_size=.5, random_state=12345)

# Investigar la calidad de diferentes modelos

In [75]:
# Seleccionar un seed aleatorio para los experimentos
random_state=12345

## Árbol de decisión

In [76]:
# Entrenar varios modelos de árbol de decisión con diferentes profundidades máximas y encontrar el mejor
best_score = 0
best_depth = 0
for depth in range(1, 50):
	model = DecisionTreeClassifier(random_state=random_state, max_depth=depth)
	model.fit(features_train, target_train)
	score = model.score(features_valid, target_valid)
	if score > best_score:
		best_score = score
		best_depth = depth
    
print("Profundidad del mejor modelo:", best_depth)
print("Exactitud del mejor modelo en el conjunto de validacion:", best_score)

Profundidad del mejor modelo: 3
Exactitud del mejor modelo en el conjunto de validacion: 0.7853810264385692


## Bosque aleatorio

In [77]:
# Entrenar varios modelos de bosque aleatorio con diferentes cantidades de estimadores y encontrar el mejor
best_score = 0
best_est = 0
for est in range(1, 50):
    model = RandomForestClassifier(random_state=random_state, n_estimators=est)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("Cantidad de estimadores del mejor modelo:", best_est)
print("Exactitud del mejor modelo en el conjunto de validacion:", best_score)

Cantidad de estimadores del mejor modelo: 23
Exactitud del mejor modelo en el conjunto de validacion: 0.7947122861586314


## Regresión logística

In [78]:
# Entrenar un modelo de regresión logística y encontrar la exactitud en el conjunto de validación
model = LogisticRegression(random_state=random_state, solver='liblinear')
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)

print("Exactitud del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)

Exactitud del modelo de regresión logística en el conjunto de entrenamiento: 0.7510373443983402
Exactitud del modelo de regresión logística en el conjunto de validación: 0.7573872472783826


## Conclusion intermedia

De los 3 tipos diferentes de clasificadores:
- El modelo de `árbol de decisión` tuvo la segunda mayor puntuación y el segundo menor tiempo de ejecución
- El modelo de `bosque aleatorio` tuvo la mayor puntuación y el mayor tiempo de ejecución
- El modelo de `regresión logística` tuvo la menor puntuación y el menor tiempo de ejecución

El modelo de **`Árbol de decisión`** con el hiperparámetro `max_depth = 3` será el modelo elegido ya que tiene el mejor balance entre puntuación y tiempo de ejecución.

# Comprobar la calidad del modelo

In [83]:
# Entrenar un modelo de árbol de decisión con la profundidad máxima encontrada anteriormente y encontrar la exactitud en el conjunto de prueba
model = DecisionTreeClassifier(random_state=random_state, max_depth=best_depth)
model.fit(features_train, target_train)
score_test = model.score(features_test, target_test)

print("Exactitud del modelo de árbol de decisión en el conjunto de prueba:", score_test)

Exactitud del modelo de árbol de decisión en el conjunto de prueba: 0.7791601866251944


## Prueba de cordura

In [85]:
# Crea un dummy classifier que siempre predice la clase más frecuente
dummy = DummyClassifier(strategy='most_frequent', random_state=random_state)
dummy.fit(features_train, target_train)

# Calcula la puntuación en los datos de prueba
dummy_score = dummy.score(features_test, target_test)

print(f'Puntuación del Dummy Classifier: {dummy_score}')

Puntuación del Dummy Classifier: 0.6842923794712286


## Conclusion intermedia

- El `Dummy Classifier`, que siempre predice la clase más frecuente, adivina correctamente el 68.4% de las veces.
- El modelo de `árbol de decisión` adivina correctamente el 77.9% de las veces. Lo que indica que el modelo está aprendiendo de los datos y realizando predicciones que son significativamente mejores que las que haría un modelo que simplemente adivina la clase más frecuente.

**Nota**: Los datos están desequilibrados, ya que un `Dummy Classifier` en un conjunto de datos equilibrado tendría una precisión del 50%.

# Conclusión general

Los modelos de clasificación probados incluyeron el `árbol de decisión`, el `bosque aleatorio` y la `regresión logística`. El análisis reveló que:

- El modelo de `árbol de decisión` tuvo la segunda mayor puntuación y el segundo menor tiempo de ejecución.
- El modelo de `bosque aleatorio` tuvo la mayor puntuación y el mayor tiempo de ejecución.
- El modelo de `regresión logística` tuvo la menor puntuación y el menor tiempo de ejecución.

El modelo de `árbol de decisión` con el hiperparámetro `max_depth = 3` fue seleccionado por su equilibrio entre puntuación '`0.779`' y tiempo de ejecución. Este modelo superó al Dummy Classifier, indicando que el modelo está aprendiendo de los datos y realizando predicciones que son significativamente mejores que un modelo que simplemente adivina la clase más frecuente.
